In [1]:
from __future__ import print_function

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.svm import SVR

from sklearn.decomposition import PCA

from sklearn.linear_model import SGDRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score

from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer

In [2]:
df = pd.read_hdf('C:\Pedro\Kaggle\TwoSigma\\train.h5')
df.head()

,id,timestamp,derived_0,derived_1,derived_2,derived_3,derived_4,fundamental_0,fundamental_1,fundamental_2,...,technical_36,technical_37,technical_38,technical_39,technical_40,technical_41,technical_42,technical_43,technical_44,y
0,10,0,0.370326,-0.006316,0.222831,-0.213030,0.729277,-0.335633,0.113292,1.621238,...,0.775208,NaN,NaN,NaN,-0.414776,NaN,NaN,-2.0,NaN,-0.011753
1,11,0,0.014765,-0.038064,-0.017425,0.320652,-0.034134,0.004413,0.114285,-0.210185,...,0.025590,NaN,NaN,NaN,-0.273607,NaN,NaN,-2.0,NaN,-0.001240
2,12,0,-0.010622,-0.050577,3.379575,-0.157525,-0.068550,-0.155937,1.219439,-0.764516,...,0.151881,NaN,NaN,NaN,-0.175710,NaN,NaN,-2.0,NaN,-0.020940
3,25,0,NaN,NaN,NaN,NaN,NaN,0.178495,NaN,-0.007262,...,1.035936,NaN,NaN,NaN,-0.211506,NaN,NaN,-2.0,NaN,-0.015959
4,26,0,0.176693,-0.025284,-0.057680,0.015100,0.180894,0.139445,-0.125687,-0.018707,...,0.630232,NaN,NaN,NaN,-0.001957,NaN,NaN,0.0,NaN,-0.007338


In [4]:
df.describe()

,id,timestamp,derived_0,derived_1,derived_2,derived_3,derived_4,fundamental_0,fundamental_1,fundamental_2,...,technical_36,technical_37,technical_38,technical_39,technical_40,technical_41,technical_42,technical_43,technical_44,y
count,1.710756e+06,1.710756e+06,1.637797e+06,1.629727e+06,1.312105e+06,1.561285e+06,1.304298e+06,1.686809e+06,1.031686e+06,1.341916e+06,...,1.708204e+06,1.691591e+06,1.691591e+06,1.690740e+06,1.708520e+06,1.666567e+06,1.690755e+06,1.706070e+06,1.473977e+06,1.710756e+06
mean,1.093858e+03,9.456257e+02,-4.536046e+00,7.729436e+11,-3.320328e-01,-5.046012e-01,1.801661e+01,-2.040938e-02,-5.703754e+08,-1.622954e-01,...,-8.584833e-02,-9.103397e-02,-8.156685e-02,-7.287001e-02,4.908321e-02,5.236218e-03,-1.699966e-02,-9.735299e-01,3.881475e-04,2.217509e-04
std,6.308563e+02,5.195685e+02,2.497382e+02,7.620606e+13,6.519810e+01,1.020749e+02,9.258360e+02,2.494859e-01,7.502322e+10,3.668149e+00,...,6.125852e-01,2.471038e-01,2.346534e-01,2.235729e-01,3.102316e-01,1.133733e-01,2.116284e-01,9.605551e-01,3.011983e-02,2.240643e-02
min,0.000000e+00,0.000000e+00,-2.017497e+04,-7.375435e-02,-9.848880e+03,-3.434176e+04,-8.551914e+03,-2.344957e+00,-1.043737e+13,-1.077101e+03,...,-1.687572e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-5.250904e-01,-4.449529e-01,-1.000000e+00,-2.000000e+00,-1.265686e-01,-8.609413e-02
25%,5.500000e+02,5.040000e+02,-1.449710e-01,-2.956479e-02,-5.967524e-02,-1.655826e-01,-1.057050e-01,-1.996543e-01,-1.960470e-01,-2.280967e-01,...,-4.050297e-01,-4.651562e-04,-1.992532e-04,-2.203252e-05,-1.521701e-01,-7.377038e-02,-3.887695e-15,-2.000000e+00,-1.998819e-02,-9.561389e-03
50%,1.098000e+03,9.560000e+02,-8.368272e-04,5.523058e-03,2.109505e-02,2.475614e-03,1.175234e-02,-4.064488e-02,-7.395084e-03,-3.029069e-02,...,-8.502064e-02,-3.951567e-12,-1.418487e-13,-1.591224e-16,-1.476793e-02,9.782702e-05,0.000000e+00,-6.597540e-01,1.117279e-05,-1.570681e-04
75%,1.657000e+03,1.401000e+03,1.199108e-01,1.078554e-01,1.952209e-01,3.037236e-01,1.556464e-01,1.303819e-01,1.832071e-01,1.764751e-01,...,1.909600e-01,-5.219879e-40,0.000000e+00,0.000000e+00,1.772415e-01,7.855728e-02,0.000000e+00,-5.188884e-08,2.047074e-02,9.520990e-03
max,2.158000e+03,1.812000e+03,3.252527e+03,1.068448e+16,3.823001e+03,1.239737e+03,6.785965e+04,1.378195e+00,5.203165e+02,7.677125e+01,...,4.957758e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.569265e+00,6.844833e-01,1.000000e+00,0.000000e+00,1.435858e-01,9.349781e-02


In [5]:
reg = SGDRegressor(n_iter=1, penalty='l2', loss='squared_loss')
reg

#svr = SVR(cache_size=1000, C=0.1, degree=4)
#svr

SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', n_iter=1, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, verbose=0, warm_start=False)

In [40]:
#tot = float(len(df['derived_1']))
#uni = float(len(df['derived_1'].unique()))
#print(tot)
#print(uni)

#print(uni / tot)

#df.apply(lambda x: len(x.unique())/len(x) if len(x.unique())/len(x) < 0.50 else None )

1710756.0
1374902.0
0.803680945734


In [6]:
df = df.fillna(0)

y = df['y'].values
X = df.drop(['id', 'y'], axis=1).values

#imp = Imputer(missing_values='NaN', strategy='mean', axis=0) # Filling nan's with mean
#imp.fit(X)
#X = imp.transform(X)

In [7]:
# Splitting data into train and cv sets using 40%, 60%
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.6, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_cv = scaler.transform(X_cv)

In [16]:
#pca = PCA(n_components='mle', svd_solver='full').fit(X_train)
pca = PCA(n_components=4, svd_solver='randomized').fit(X_train)
X_train_pca = pca.transform(X_train)
X_cv_pca = pca.transform(X_cv)

print("Number of Components: %d" % pca.n_components_)

reg.fit(X_train_pca,y_train) # Remove NaN's before fitting data
#svr.fit(X_train_pca,y_train) # Remove NaN's before fitting data

Number of Components: 4


SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', n_iter=1, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, verbose=0, warm_start=False)

In [17]:
print(reg.predict(X_cv_pca)[:20])
#print(svr.predict(X_cv_pca)[:20])
print(y_cv[:20])

[  1.65346226e+09   6.22860155e+08   2.40382974e+07   1.97944359e+09
  -6.60968248e+07   3.33472741e+08   9.21029695e+08  -1.07382433e+08
   1.18945500e+09  -1.55343005e+08   4.63782991e+08  -8.57723166e+08
  -1.03560439e+09   1.86817660e+09  -9.58863253e+08  -8.19233376e+08
   1.12421960e+09   4.25194342e+08   2.56757664e+09   4.85485703e+08]
[-0.05937321 -0.0079669  -0.00313469 -0.01286841 -0.01544089 -0.00391624
 -0.02095961 -0.00787476 -0.00264549 -0.01243801  0.02336292  0.05032206
 -0.00117565 -0.03457598 -0.01040553  0.00349346 -0.02084679 -0.0292836
 -0.02136473  0.02028296]


In [133]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - An object to be used as a cross-validation generator.
          - An iterable yielding train/test splits.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : integer, optional
        Number of jobs to run in parallel (default 1).
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt


title = "Learning Curves - SGD"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

plot_learning_curve(reg, title, X_train_pca, y_train, cv=cv, n_jobs=4)

plt.show()

In [19]:
rate_param = np.logspace(-5,-4,num=5,base=10.0)
#rate_param = 10.0**-np.arange(1,7)
print(rate_param)

#tuned_parameters = [{'alpha': rate_param, 'loss': ['huber','squared_loss'], 'penalty': ['l1','l2']}] 
#tuned_parameters = [{'epsilon': rate_param}]
tuned_parameters = [{'alpha': rate_param}]
#tuned_parameters = [{'degree': np.arange(4,15), 'C': np.logspace(-1,0,num=10,base=10.0)}]

scores = ['neg_mean_squared_error']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()   

    gscv = GridSearchCV(reg, tuned_parameters, cv=5, scoring='%s' % score)
    #gscv = GridSearchCV(svr, tuned_parameters, cv=5, scoring='%s' % score)
    gscv.fit(X_train_pca, y_train)

    print("Best parameters set found on development set:")
    print()
    print(gscv.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = gscv.cv_results_['mean_test_score']
    stds = gscv.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, gscv.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()

    print("Detailed regression report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    
    y_true, y_pred = y_cv, gscv.predict(X_cv_pca)
    
    print(mean_squared_error(y_true,y_pred))
    print()

[  1.00000000e-05   1.08263673e-05   1.17210230e-05   1.26896100e-05
   1.37382380e-05   1.48735211e-05   1.61026203e-05   1.74332882e-05
   1.88739182e-05   2.04335972e-05   2.21221629e-05   2.39502662e-05
   2.59294380e-05   2.80721620e-05   3.03919538e-05   3.29034456e-05
   3.56224789e-05   3.85662042e-05   4.17531894e-05   4.52035366e-05
   4.89390092e-05   5.29831691e-05   5.73615251e-05   6.21016942e-05
   6.72335754e-05   7.27895384e-05   7.88046282e-05   8.53167852e-05
   9.23670857e-05   1.00000000e-04]
# Tuning hyper-parameters for neg_mean_squared_error

Best parameters set found on development set:

{'alpha': 2.592943797404667e-05}

Grid scores on development set:

-26104094438793408413696.000 (+/-43491304439182729936896.000) for {'alpha': 1.0000000000000001e-05}
-9660334779485416062976.000 (+/-13567711688061693722624.000) for {'alpha': 1.0826367338740541e-05}
-6163400598307641229312.000 (+/-21258789907642412171264.000) for {'alpha': 1.1721022975334793e-05}
-54566884664916